In [ ]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [ ]:
import os

import geopandas as gpd
import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo
from sqlalchemy import text

In [ ]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(admin=True)

# TEOTIL3: Tidy annual data

## Part 1: Update administrative boundaries

Fylke and kommune boundaries may change from year to year. This notebook makes sure the latest administrative names and codes are present in the TEOTIL3 database and assigns regines to administrative units.

## Workflow overview

 1. Download the latest **kommune boundaries** for the **current year** from [GeoNorge](https://www.geonorge.no/). Choose UTM Zone 33N as the CRS and FGDB as the output format. Fylke boundaries are derived by dissolving the kommune dataset, so it is not necessary to download the fylke dataset as well.
    
 2. Rename the FGDB `kommuner{year}.gdb`. Zip it, upload to `/home/jovyan/shared/common/teotil3/core_data/administrative/raw`, unzip, and delete the zip archive.

 3. Run the code cells below to process the new administrative data and assign it to regines. **Note that the NIBIO regines are hard-coded below (`nib_gpkg`)**. This is because NIBIO cannot update their regine dataset each year.
 
 4. Run the code cell to upload data to PostGIS.

In [ ]:
# Data are being uploaded for 'data_year'
data_year = 2023

# 'admin_year' is usually the current year (i.e. 'data_year' + 1).
# It's also the year for which new admin. boundaries will be added.
admin_year = data_year + 1

## 1. Process data

In [ ]:
# Process admin. boundaries
admin_data_fold = r"/home/jovyan/shared/common/teotil3/core_data/administrative"
admin_gpkg = os.path.join(admin_data_fold, "admin_data.gpkg")
gdb_path = os.path.join(admin_data_fold, "raw", f"kommuner{admin_year}.gdb")
gdf = gpd.read_file(gdb_path, layer="kommune")

# Get fylkesnummer from kommnr
gdf.rename({"kommunenummer": "komnr"}, axis="columns", inplace=True)
gdf["komnr"] = gdf["komnr"].astype(str)
gdf["fylnr"] = gdf["komnr"].str[:2]
gdf = gdf[["fylnr", "komnr", "geometry"]]

# Dissolve
kom_gdf = gdf.dissolve(by="komnr", aggfunc="first").reset_index()
kom_gdf = kom_gdf[["fylnr", "komnr", "geometry"]]
fyl_gdf = gdf.dissolve(by="fylnr").reset_index()
fyl_gdf = fyl_gdf[["fylnr", "geometry"]]

# Save
kom_gdf.to_file(
    admin_gpkg,
    driver="GPKG",
    layer=f"kommuner{admin_year}",
    index=False,
)
fyl_gdf.to_file(
    admin_gpkg,
    driver="GPKG",
    layer=f"fylker{admin_year}",
    index=False,
)

# Assign to regines
nib_gpkg = r"/home/jovyan/shared/common/teotil3/core_data/hydrology/nibio_regines.gpkg"
reg_gdf = gpd.read_file(nib_gpkg, layer="RegineEnhetNibio")
reg_gdf = teo.preprocessing.assign_regines_to_administrative_units(
    reg_gdf, admin_gpkg, admin_year
)
reg_gdf.rename(
    {"komnr": f"komnr_{admin_year}", "fylnr": f"fylnr_{admin_year}"},
    axis="columns",
    inplace=True,
)
reg_gdf = reg_gdf[["regine", f"komnr_{admin_year}", f"fylnr_{admin_year}"]]
assert reg_gdf.isna().sum().sum() == 0

## 2. Add to PostGIS

In [ ]:
# # Write to temp table
# reg_gdf.to_sql(
#     "temp",
#     eng,
#     schema="teotil3",
#     index=False,
#     if_exists="replace",
# )

# # Extend existing table with new info
# with eng.connect() as connection:
#     with connection.begin() as transaction:
#         # Add new columns
#         sql = text(
#             f"ALTER TABLE teotil3.regines "
#             f"ADD COLUMN komnr_{admin_year} text, "
#             f"ADD COLUMN fylnr_{admin_year} text"
#         )
#         connection.execute(sql)

#         # Update with data from temporary table
#         sql = text(
#             f"UPDATE teotil3.regines "
#             f"SET komnr_{admin_year} = temp.komnr_{admin_year}, "
#             f"    fylnr_{admin_year} = temp.fylnr_{admin_year} "
#             f"FROM teotil3.temp temp "
#             f"WHERE teotil3.regines.regine = temp.regine"
#         )
#         connection.execute(sql)

#         # Add NOT NULL constraint
#         sql = text(
#             f"ALTER TABLE teotil3.regines "
#             f"ALTER COLUMN komnr_{admin_year} SET NOT NULL, "
#             f"ALTER COLUMN fylnr_{admin_year} SET NOT NULL"
#         )
#         connection.execute(sql)

#         # Drop the temporary table
#         sql = text("DROP TABLE teotil3.temp")
#         connection.execute(sql)

#         transaction.commit()